In [2]:
%load_ext autoreload
%autoreload 2


import pandas as pd

import spacy

import random

import numpy as np

from os import getcwd
from os.path import join, dirname

PATH_REPO = dirname(getcwd())
PATH_UTILS = join(PATH_REPO, 'utils')
PATH_DATA = join(PATH_REPO, 'data')
PATH_MODELS = join(PATH_REPO, 'models')


nlp = spacy.load('en_core_web_lg')


import sys
sys.path.append(PATH_UTILS)

pd.set_option('max_colwidth', None)

random.seed(1)
spacy.util.fix_random_seed(1)



from sklearn.metrics import precision_recall_fscore_support as score
import _pickle as cPickle
from training import create_word2vec_dataset


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
test = pd.read_csv(join(PATH_DATA, 'dev_set_proc.csv'), names=['text', 'label'], skiprows=1)
# test['label'] = np.where(test['label']=='Gratitude', 1, 0)
test

,text,label
0,USER Thank you for the reply! I appreciate the help :) the book is The Art Of Thor: Ragnarok:,1
1,USER Thank you for the reply Goldie. It's not just sainsburys ... I haven't seen them anywhere this year in supermarkets. Not even in the mixed nut bags :)) Appreciate your tweet though. Have a good weekend :),1
2,USER Appreciate it. We love USER and only want to see her succeed in the future... so appreciate you letting me be your (somewhat worried) canary in the coal mine ;),1
3,"USER Thanks so much. I meant to tweet not message, so I appreciate the quick reply!! Enjoy your weekend. :-)",1
4,USER USER Dear USER USER USER thank you for taking action and getting back to me. Case has been solved thank you for your quick reply. Happy Weekend :),1
...,...,...
608,USER We'd hate to see you go! We understand the frustration and will share your feedback with our team.,0
609,"USER We would be too! We're so sorry to hear about the trouble you've been having, Tim. What device are you on? Running into errors?",0
610,"USER Terribly sorry for the agitation regarding your husband's account, Ailsa! What information were we able to provide when you contacted us about it? ^JZ",0
611,USER The 17:36 was cancelled due to a fault and 17:49 because of a broken down train earlier. Apologies for this and for any problems they have caused. Lewis,0


In [4]:
X_test = create_word2vec_dataset(test, nlp, False)

In [5]:
with open(join(PATH_MODELS,'gratitude_classifier.pkl'), 'rb') as fid:
    classifier = cPickle.load(fid)

In [6]:
predictions = classifier.predict(X_test)
test['prediction'] = predictions
Y_test = test['label'].values.tolist()
test.head()

,text,label,prediction
0,USER Thank you for the reply! I appreciate the help :) the book is The Art Of Thor: Ragnarok:,1,1
1,USER Thank you for the reply Goldie. It's not just sainsburys ... I haven't seen them anywhere this year in supermarkets. Not even in the mixed nut bags :)) Appreciate your tweet though. Have a good weekend :),1,1
2,USER Appreciate it. We love USER and only want to see her succeed in the future... so appreciate you letting me be your (somewhat worried) canary in the coal mine ;),1,1
3,"USER Thanks so much. I meant to tweet not message, so I appreciate the quick reply!! Enjoy your weekend. :-)",1,1
4,USER USER Dear USER USER USER thank you for taking action and getting back to me. Case has been solved thank you for your quick reply. Happy Weekend :),1,1


## False positives

In [7]:
test[(test['label'] == 0) & (test['prediction'] == 1)][50:100]

,text,label,prediction
418,when you pay for next day delivery for coats you need for kids tomorrow and they do not arrive! Not a happy mummy! USER,0,1
419,"USER why do you suck? Tried buying things today and paying with paypal, dont work and no explanation. Great service...",0,1
434,USER Suffice it to say I am not happy and I lost money I can't get refunded (not to mention missing a tour that will not be repeated).,0,1
435,"USER USER As discussed, Rather than your POOR REPETITIVE execuses and causing more agony. Please cancel the order. You ensure all possible ways to cause inconvenience. Unfortunately I am still struck with you due to Amazon Pay Balance. My bad and hard luck.",0,1
437,"USER It was at Hinckley but they were too busy to bother... not interested but for £6, i was.",0,1
441,USER I wish I got a reminder about £39 being taken out of my account nearly cried when I saw it 😢😅 Poor,0,1
446,"Dear USER store 86 in Nashville has the worst service &amp; your call center, to which I was directed sucks too",0,1
454,USER I missed really important meetings and almost lost my job because of your app. You also steal money from customers.,0,1
461,USER USER USER That's really disappointing! As it seems like I'll be getting home after 9pm this evening which is really unfair!!!,0,1
467,Fuck me my internet just died thanks USER,0,1


## False negatives

In [8]:
test[(test['label'] == 1) & (test['prediction'] == 0)]

,text,label,prediction
75,So thankful everything I get at USER is gluten free. Makes not being able to eat gluten suck a little less ♥,1,0
82,USER You’re the best ♥️,1,0


## True positives

In [9]:
test[(test['label'] == 1) & (test['prediction'] == 1)]

,text,label,prediction
0,USER Thank you for the reply! I appreciate the help :) the book is The Art Of Thor: Ragnarok:,1,1
1,USER Thank you for the reply Goldie. It's not just sainsburys ... I haven't seen them anywhere this year in supermarkets. Not even in the mixed nut bags :)) Appreciate your tweet though. Have a good weekend :),1,1
2,USER Appreciate it. We love USER and only want to see her succeed in the future... so appreciate you letting me be your (somewhat worried) canary in the coal mine ;),1,1
3,"USER Thanks so much. I meant to tweet not message, so I appreciate the quick reply!! Enjoy your weekend. :-)",1,1
4,USER USER Dear USER USER USER thank you for taking action and getting back to me. Case has been solved thank you for your quick reply. Happy Weekend :),1,1
...,...,...,...
340,"USER No I get it, just thought I complain a little cause when I signed up for “limited ads” I thought maybe 3-4 every once in awhile not 2 long ads in a row. But I appreciate you responding! :)",1,1
343,"USER That's a shame, thanks for looking though! That color combo is what I'm looking for though. Ah well ;) appreciate the attempt!",1,1
346,"USER Thanks, but I've already checked and there is not a single store in my country, I appreciate the effort :D",1,1
347,"USER Of course, as someone who also works in customer success I️ can appreciate it’s the little things that go a long way and everyone on this flight is taking notice of the tone Greg is setting:)",1,1


## True negatives

In [10]:
len(test[(test['label'] == 0) & (test['prediction'] == 0)])

154

In [11]:
precision, recall, fscore, support = score(Y_test, predictions, average='macro')
print("Precision:\t", precision, "\n", "Recall:\t", recall, "\n", "F1:\t", fscore)

Precision:	 0.8010763746057864 
 Recall:	 0.7540789169173803 
 F1:	 0.7187450022535293
